Importation des bibliothèques et Chargement des données nettoyées

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
project_root = r'C:\Users\Marouane\ImmoPredict'
processed_path = os.path.join(project_root, 'data', 'processed', 'dvf_clean.csv')
df = pd.read_csv(processed_path)
print(f"Dimensions initiales: {df.shape}")

C:\Users\Marouane\AppData\Local\Temp\ipykernel_12428\3818663381.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(processed_path)


Dimensions initiales: (586349, 27)


Filtrage pour se concentrer sur les appartements

In [5]:
if 'Type local' in df.columns:
    df = df[df['Type local'] == 'Appartement'].copy()
    print(f"Dimensions après filtrage des appartements: {df.shape}")

2. Création de variables dérivées

In [6]:

# Prix au m²
if all(col in df.columns for col in ['Valeur fonciere', 'Surface reelle bati']):
    # Éviter la division par zéro
    mask = df['Surface reelle bati'] > 0
    df.loc[mask, 'Prix_m2'] = df.loc[mask, 'Valeur fonciere'] / df.loc[mask, 'Surface reelle bati']
    
    # Filtrer les prix au m² aberrants
    df = df[(df['Prix_m2'] > 100) & (df['Prix_m2'] < df['Prix_m2'].quantile(0.99))]

# Surface par pièce
if all(col in df.columns for col in ['Surface reelle bati', 'Nombre pieces principales']):
    mask = df['Nombre pieces principales'] > 0
    df.loc[mask, 'Surface_par_piece'] = df.loc[mask, 'Surface reelle bati'] / df.loc[mask, 'Nombre pieces principales']

# Variables polynomiales
if 'Surface reelle bati' in df.columns:
    df['Surface_squared'] = df['Surface reelle bati'] ** 2



3. Encodage des variables catégorielles

In [7]:

# One-hot encoding pour les variables catégorielles
categorical_cols = ['Code departement']
if 'Type local' in df.columns and len(df['Type local'].unique()) > 1:
    categorical_cols.append('Type local')

for col in categorical_cols:
    if col in df.columns:
        dummies = pd.get_dummies(df[col], prefix=col.replace(' ', '_'))
        df = pd.concat([df, dummies], axis=1)



4. Agrégation de variables géographiques

In [8]:

# Prix moyen par département
if 'Code departement' in df.columns:
    dept_price = df.groupby('Code departement')['Valeur fonciere'].mean().reset_index()
    dept_price.columns = ['Code departement', 'Prix_moyen_dept']
    df = df.merge(dept_price, on='Code departement', how='left')
    
    # Ratio prix/prix moyen du département
    df['Ratio_prix_dept'] = df['Valeur fonciere'] / df['Prix_moyen_dept']



5. Suppression des colonnes non nécessaires pour la modélisation

In [9]:

# Colonnes à supprimer (ajustez selon vos besoins)
cols_to_drop = ['Type local', 'Code departement']  # Nous avons déjà créé des variables encodées
cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df = df.drop(columns=cols_to_drop)


6. Gestion des valeurs manquantes restantes

In [10]:

# Affichage des valeurs manquantes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Nombre': missing_values, 'Pourcentage (%)': missing_percent})
print("\nValeurs manquantes par colonne:")
display(missing_df[missing_df['Nombre'] > 0].sort_values('Pourcentage (%)', ascending=False))

# Imputation des valeurs manquantes
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())


Valeurs manquantes par colonne:


,Nombre,Pourcentage (%)
2eme lot,580444,100.000000
Surface Carrez du 1er lot,579287,99.800670
1er lot,578692,99.698162
Type de voie,82205,14.162434
No voie,1727,0.297531
Surface_par_piece,956,0.164702
Code postal,45,0.007753
Section,23,0.003962
Voie,4,0.000689


c:\Users\Marouane\ImmoPredict\venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


7. Sauvegarde du dataset avec les features


In [11]:

features_path = os.path.join(project_root, 'data', 'processed', 'dvf_features.csv')
df.to_csv(features_path, index=False)
print(f"\nDataset avec features sauvegardé dans: {features_path}")
print(f"Dimensions finales: {df.shape}")


Dataset avec features sauvegardé dans: C:\Users\Marouane\ImmoPredict\data\processed\dvf_features.csv
Dimensions finales: (580444, 130)
